# Sense HAT for PYNQ:Character display

This notebook illustrates how to interact with the [Sense HAT](https://www.raspberrypi.org/products/sense-hat/) and display character on the LED matrix of Sense HAT.


This example notebook includes the following steps.
1. import python libraries
2. select RPi switch and using Microblaze Library
3. configure the I2C device
4. convert characters
5. waiting for user's input and display on the Sense HAT LED matrix

![PYNQ with Sense HAT](data/PYNQ_with_Sense_HAT.jpg)


### 1. Sense HAT Introduction

The Sense HAT, which is a fundamental part of the [Astro Pi](https://astro-pi.org/) mission, allows your board to sense the world around it.It has a 8×8 RGB LED matrix, a five-button joystick and includes the following sensors:

* Gyroscope
* Accelerometer
* Magnetometer
* Temperature
* Barometric pressure
* Humidity

![Sense HAT add-on board](data/Sense_HAT.jpg)


### 2. Prepare the overlay
Download the overlay first, then select the shared pin to be connected to
RPi header (by default, the pins will be connected to PMODA instead).

In [10]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
from PIL import Image, ImageDraw, ImageFont, ImageShow, ImageColor 
import time
import numpy as np

In [11]:
base = BaseOverlay('base.bit')
lib = MicroblazeLibrary(base.RPI, ['i2c','gpio', 'xio_switch','circular_buffer'])

### 3. Configure the I2C device and GPIO device
Initialize the I2C device and set the I2C pin of RPi header. Since the PYNQ-ZU board does not have pull-up on the Reset_N pin of the HAT (GPIO25), set that to 1. 

In [12]:
device = lib.i2c_open_device(1)
lib.set_pin(2, lib.SDA1)
lib.set_pin(3, lib.SCL1)
gpio25=lib.gpio_open(25)
lib.gpio_set_direction(gpio25, 0);
gpio25.write(1)

In [13]:
LED_MATRIX_ADDRESS = 0x46

### 4. Convert characters
Render characters and organize images into dictionary

In [15]:
text = ' +-*/!"><0123456789.=)(ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz?,;:\''
img = Image.new('RGB', (len(text)*5, 8), color = (0,0,0))
d = ImageDraw.Draw(img)
d.text((0,-1), text, fill=(255,255,255),font=ImageFont.truetype("/usr/share/fonts/truetype/ttf-bitstream-vera/VeraMono.ttf",8))
text_pixels = list(map(list, img.rotate(-90, expand=True).getdata()))

In [16]:
text_dict = {}
for index, s in enumerate(text):
    start = index * 40
    end = start + 40
    char = text_pixels[start:end]
    text_dict[s] = char

### 5. Display
Wait for input and display on the LED matrix

In [17]:
buf = bytearray(8*8*3+1) # Display size 8x8 grid (3 color) plus end char
buf[0] = 0

In [19]:
done = False
while not done:
    display = input("Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminate.\n")
    if(display == "c" or display == "C" ):
        done = True
    else:
        display = display + " "
        for value in display:
            for i in range(0,len(buf)) :
                buf[i] = 0;
            for y in range(0,8) :
                for x in range(2,7) :
                    buf[1+x+8*0+3*8*y] = int(text_dict[value][8*(6-x)+(y)][0]/20); #R
                    buf[1+x+8*1+3*8*y] = int(text_dict[value][8*(6-x)+(y)][1]/20); #G
                    buf[1+x+8*2+3*8*y] = int(text_dict[value][8*(6-x)+(y)][2]/20); #B
            lib.i2c_write(device, LED_MATRIX_ADDRESS, buf, len(buf))
            time.sleep(0.5)

Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminate.
'HELLO,XILINX!'
Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminate.
"Hello"
Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminate.
"HELLO,XILINX!"
Please input a string(e.g.'HELLO,XILINX!') and press enter. Input 'C' to terminate.
c


In [20]:
device.close()

Copyright (C) 2020 Xilinx, Inc